In [1]:
import numpy as np

# Calculating Error with Loss

Ahhoz, hogy tanítani tudjuk a modelt tudnunk kell számszerűsíteni, hogy mennyire rossz a modell.

Szükségünk van egy olyan módszerre, amivel kiszámíthatjuk, hogy a neurális hálózat mennyire téved az előrejelzésekben, hogy elkezdhessük a súlyok és a torzítások beállítását, hogy csökkentsük a hibát.

A modell betanításához a súlyokat és a torzításokat úgy módosítjuk, hogy javítsuk a modell pontosságát, azaz csökkentsük a hiba mértékét. A veszteségfüggvény az az algoritmus, ami számszerűsíti a modell tévedését. A veszteség (loss) ennek a mérőszámnak a mértéke. Mivel a veszteség a modell hibája, ideális esetben értéke 0.

A neurális hálózat kimenete mindig egy "bizalom", és a helyes válaszban való nagyobb bizalom jobb. Arra törekszünk, hogy növeljük a helyes bizalmat és csökkentsük a helytelen bizalmat.

Több féle veszteség függvény létezik, mindig a célhoz választjuk őket. 

Néhány művelet mindegyikben közös. Az egyik ilyen művelet a teljes veszteség kiszámítása, a teljes veszteség mindig az összes minta veszteségének átlagértéke.

(Például a regressziót végző neurális hálózatoknál használt veszteségfüggvények a négyzetes hiba (átlagos négyzetes hiba).)

A veszteség hasznos mérőszám a modell optimalizálásához, a gyakorlatban a veszteséggel együtt használt mérőszám a pontosság, amely azt írja le, hogy a legnagyobb bizalom milyen gyakran a helyes osztály.

<br>

## Categorical Cross-Entropy Loss

Osztályozás feladathoz használatos veszteség függvény.

A modell egy softmax aktiválási függvényt használ a kimeneti réteghez, vagyis egy valószínűségi eloszlást ad ki. 

A kategorikus kereszt-entrópiát egy alapigazság ("targets", $y$) és valamilyen megjósolt eloszlás ("predictions", $\hat{y}$) összehasonlítására használják.

Az $y$ (tényleges/kívánatos eloszlás) és az $\hat{y}$ (előre jelzett eloszlás) kategorikus kereszt-entrópiájának kiszámítására szolgáló képlet:

$$\Huge
L_i=-\sum_j y_{i, j} \log \left(\hat{y}_{i, j}\right)
$$


- $L_i$: a minta veszteségértéke 
- $i$: $i$-edik minta a halmazban
- $j$: címke/kimeneti index,
- $y$: célértékek
- $\hat{y}$: az előre jelzett érték


**A kereszt-entrópia két valószínűségi eloszlást hasonlít össze.**

A kívánt valószínűségi eloszlás: `[1, 0, 0]` (tudjuk, hogy az első kategória a helyes)

Az ilyen vektorokat one-hot-nak nevezzük, mivel az egyik érték "forró" (on), 1 értékkel, a többi pedig "hideg" (off), 0 értékkel. 

Amikor a modell eredményeit egy one-hot vektorral hasonlítjuk össze a kereszt-entrópia segítségével, az egyenlet többi része lenullázódik, és a célvalószínűség log veszteségét megszorozzuk 1-gyel, így a kereszt-entrópia számítása nagyon egyszerű. 

Mondjuk a modellünk softMax kimenete: `[0.7, 0.1, 0.2]`


\begin{aligned}
& L_i=-\sum_j y_{i, j} \log \left(\hat{y}_{i, j}\right)=-(1 \cdot \log (0.7)+0 \cdot \log (0.1)+0 \cdot \log (0.2))= \\
& =-(-0.35667494393873245+0+0)=0.35667494393873245
\end{aligned}


Az eredmény bizalmi szintje lehet például `[0.22, 0.6, 0.18]` vagy `[0.32, 0.36, 0.32]` 

Mindkét esetben a modell a második osztályt adja vissza előrejelzésként, de a modell bizalma csak az első esetében magas. A kategorikus kereszt-entrópia veszteség ezt figyelembe veszi, és **annál nagyobb veszteséget ad ki, minél kisebb a bizalom.** A veszteség értéke a megbízhatósági szinttel együtt nő, és megközelíti a 0-t (`log(0)` nincsen)

In [2]:
softmax_output = [0.7, 0.1, 0.2]   # ezt adja a háló 
target_output = [1, 0, 0]          # ez lenne az ideális (one-hot vector)

In [3]:
loss = -(np.log(softmax_output[0]*target_output[0] + 
                softmax_output[1]*target_output[1] + 
                softmax_output[2]*target_output[2]))

In [4]:
loss

0.35667494393873245

Tehát ez a hiba nagysága, minél kissebb annál jobb.

---

### Batch 

Több eredményt akarunk egyszerre kiértékelni.

In [5]:
# batch=3 output
softmax_outputs = np.array([[0.7, 0.1, 0.2],
                            [0.1, 0.5, 0.4],
                            [0.02, 0.9, 0.08]])  

A példában 3 osztály van: valamit "kutya", "macska" vagy "ember" kategóriába próbálunk besorolni. 
- kutya 1. osztály (0. index)
- macska 2. osztály (1. index)
- ember 3. osztály (2. index)

Tegyük fel, hogy a helyes válaszok: *kutya, macska, macska* (első kép kutya, második macsak, harmadik macska).

tehát a célérték index lista:

In [6]:
class_targets = [0, 1, 1] 

In [7]:
# a helyes válaszokhoz tartozó valószínűségi értékek:
softmax_outputs[[0, 1, 2], class_targets]

array([0.7, 0.5, 0.9])

_Minden sor kell, de mindegyikből csak a helyes válasz. A nullás sornak kérem a nullás elemét, az egyes sornak kérem az egyes elemét, a kettes sornak kérem az egyes elemét._

In [8]:
# dinamikus megoldás
softmax_outputs[np.arange(len(softmax_output)), class_targets]

array([0.7, 0.5, 0.9])

A helyes megoldásokon kiszámolom a loss értékeket:

In [9]:
loss = -(np.log(softmax_outputs[np.arange(len(softmax_output)), class_targets]))

In [10]:
loss

array([0.35667494, 0.69314718, 0.10536052])

Batch átlagos hibája:

In [11]:
np.mean(loss)

0.38506088005216804

---

Belefuthatunk egy problémába: a 0-nak nem tudjuk kiszámolni log-ját mivel az végtelen. Végtelen objektummal pedig nem tud dolgozni a többi függvény, de van a numpy-ban egy beépített függvény ami `inf` objektum helyett egy beállított legkisebbb és legnagyobb számot ad vissza.

Ez megakadályozza, hogy a veszteség pontosan 0 legyen, ehelyett egy nagyon kis érték lesz, de nem lesz negatív érték, és nem torzítja a teljes veszteséget 1 felé.

In [12]:
np.clip(np.log(0), 1e-7, 1 - 1e-7)

/var/folders/62/vz7ys27s3yj5g2sggr7qwbkr0000gn/T/ipykernel_17965/456330982.py:1: RuntimeWarning: divide by zero encountered in log
  np.clip(np.log(0), 1e-7, 1 - 1e-7)


1e-07

---

#### One-hot vector

A célpontok eredményei általában one-hot vector kódolásúak, tehát nem egy külön listában vannak a helyes eredmények indexei, hanem minden mintához van egy külön vektor, ami a helyes válasz pozicióját mutatja:

In [13]:
class_targets = np.array([[1, 0, 0],
                          [0, 1, 0],
                          [0, 1, 0]])

Ebben az esetben a célcímkéknél a valószínűségek kiszűrése helyett meg kell szoroznunk a valószínűséget a célcímkékkel, **a helyes címkéknél lévő értékek kivételével minden értéket lenullázva**, majd a sortengely (1. tengely) mentén összegzést végezve megkapjuk a helyes válasz valószínűségét:

In [14]:
softmax_outputs*class_targets

array([[0.7, 0. , 0. ],
       [0. , 0.5, 0. ],
       [0. , 0.9, 0. ]])

In [15]:
np.sum(softmax_outputs*class_targets, axis=1)

array([0.7, 0.5, 0.9])

---

## Veszteség számoló osztály

Több veszteségfüggvényünk is lesz, és néhány művelet, ami mindegyikben közös (például a teljes veszteség kiszámítása átlagolással).

Ezért lesz egy fő veszteség osztályunk majd ebből csináljuk meg a speciális osztályokat a konkrét számolással.

In [16]:
class Loss:
    
    def calculate(self, output, y):
        sample_losses = self.forward(output, y) # output: argmax output, y: traning data
        data_loss = np.mean(sample_losses)
        return data_loss


class Loss_CCE(Loss):

    def forward(self, y_pred, y_true): # y_pred: argmax output, y_true: traning data
        samples = len(y_pred) # number of samples in a batch
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        # el kell dönteni, hogy lista vagy one-hot vektorok vannak-e
        if len(y_true.shape) == 1: # categorical labels (index szűrés megoldás)
            correct_confidences = y_pred[range(samples), y_true]
            
        elif len(y_true.shape) == 2: # one-hot encoded labels (helytelen nullázás megoldás)
            correct_confidences = np.sum(y_pred*y_true, axis=1)

        # Loss számolás
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

---

#### Teszt

In [17]:
softmax_outputs = np.array([[0.7, 0.1, 0.2],
                            [0.1, 0.5, 0.4],
                            [0.02, 0.9, 0.08]])

onehot_targets = np.array([[1, 0, 0],
                           [0, 1, 0],
                           [0, 1, 0]])

index_targets = np.array([0, 1, 1])

In [18]:
loss_fugveny = Loss_CCE()

In [19]:
# one-hot vektor bemenet
loss_fugveny.calculate(softmax_outputs, onehot_targets)

0.38506088005216804

In [20]:
# index bemenet
loss_fugveny.calculate(softmax_outputs, index_targets)

0.38506088005216804

In [21]:
np.argmax(softmax_outputs, axis=1)

array([0, 1, 1])

---

Ellenőrzés, hogy a modell hány bemenet talált el helyesen:

In [22]:
predictions = np.argmax(softmax_outputs, axis=1) # a legnagyobb érték indexe (soronként)

In [23]:
predictions == index_targets

array([ True,  True,  True])

In [24]:
accuracy = np.mean(predictions == index_targets) # 2/3 (háromból 2 helyes)

In [25]:
accuracy

1.0

---

One-hot vector átlakítása index listává:

In [26]:
np.argmax(onehot_targets, axis=1)

array([0, 1, 1])